In [1]:
import pandas as pd
import requests
import json
import glob
import os

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMC0wNy0xMVQwNzo1Mjo0Mi40NjBaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE1OTQ0NTM5NjIsImV4cCI6MTYwMTc5Nzk2MiwiYXRfaGFzaCI6InRESkRxZUJnRWpBa2N5dFNKUjdCZkEiLCJub25jZSI6IkFmTy5YSn5ZfklOaHh5M2dUZ0hyTC50QUdoRmt6ZWJjIn0.P01TNTizPXHobT3Yt8Md3gTUNkK7I9opZMilZ_iAHsU6-YHwuRQo0jjIdyqtsTvxhBzGpCsEeEez7h2C1kOnrsIF57m3O6DRnRoEeNyB52vP--z7A-zx1KrY2EuawqAVeWc5-mQCcYohSWmx75yhi0bQD3PoPbP6jNxUnmYT3O11_AbnZAM5UkB6jaAcYpsfUVXl6cB_z_R1iuWJvt5fSHJg53LPtL4waV61YEaVCbl8h9ON1Cz13-vsd_2dndxwC_CqA9gdLEw2HUF195ebtDMvVN6raYojvRZGg1EI9HVcCK0sP4q6VUspGDMZm2L97UaKCw7SYwA5zQ6WJ46bJw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"544e22ddb86b281730765549eeb46b2dcb028fce","refresh_token":"870a489b48554fbf5d8845d106c9cac5d3d0f630"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,151,293957,Brodie,Grundy,COL,RUC
0,469,290528,Max,Gawn,MEL,RUC
0,755,295467,Jack,Macrae,WBD,MID
0,76,293535,Lachie,Neale,BRL,MID
0,244,291570,Nat,Fyfe,FRE,MID
...,...,...,...,...,...,...
0,818,1016214,Ayce,Taylor,ADE,DEF
0,821,1016097,Henry,Crauford,ESS,RUC
0,822,1020224,Tom,Hird,ESS,MID
0,825,297668,Nicholas,Reid,WCE,FWD


## LADDER AND FIXTURE DATA

In [5]:
# specify round
round_number = 10
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)

## COACH IDs AND NAMES

In [6]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,9806,Lester,HoweIMetYourMother
0,180650,33301,Jordan,"What's Up, Doch?"
0,45168,5524,Luke,The Dragon's Laird
0,300791,182,Paul,Coronaviney
0,21562,186,Mark,GetRichOrDieTryin
0,10857,189,Anthony,The Full Bonty
0,71052,9867,James,Bad Boys For Fyfe
0,8258,9856,Simon,It's always Sonny


## PLAYER MATCH RESULTS

Function to get player scores for each match

In [7]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
#     if on_field_status == True:
#         scoring_tag = 'scoring'
#     else:
#         scoring_tag = 'nonscoring'
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [8]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,86078,9806,Lester,HoweIMetYourMother,153,DEF,156,True,291313,Jeremy,Howe,COL,DEF,R1
1,2,86078,9806,Lester,HoweIMetYourMother,153,DEF,126,True,291313,Jeremy,Howe,COL,DEF,R2
2,3,86078,9806,Lester,HoweIMetYourMother,153,DEF,88,True,291313,Jeremy,Howe,COL,DEF,R3
3,4,86078,9806,Lester,HoweIMetYourMother,153,DEF,112,True,291313,Jeremy,Howe,COL,DEF,R4
4,1,86078,9806,Lester,HoweIMetYourMother,789,DEF,152,True,240283,Shannon,Hurn,WCE,DEF,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,10,21562,186,Mark,GetRichOrDieTryin,770,DEF,105,True,992351,Bailey,Williams,WBD,DEF,R10
1756,10,21562,186,Mark,GetRichOrDieTryin,528,DEF,95,True,296355,Luke,McDonald,NTH,DEF,R10
1757,10,21562,186,Mark,GetRichOrDieTryin,580,MID,63,True,293603,Sam,Mayes,PTA,MID,R10
1758,10,21562,186,Mark,GetRichOrDieTryin,549,FWD,49,True,998484,Cameron,Zurhaar,NTH,FWD,R10


In [9]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [10]:
df_player_match_results = pd.DataFrame()

df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,86078,9806,Lester,HoweIMetYourMother,1,0,0,1827,1476,4,1
0,1,180650,33301,Jordan,"What's Up, Doch?",1,0,0,1721,1623,4,2
0,1,45168,5524,Luke,The Dragon's Laird,1,0,0,1720,1662,4,3
0,1,300791,182,Paul,Coronaviney,1,0,0,1686,1657,4,4
0,1,21562,186,Mark,GetRichOrDieTryin,0,0,1,1657,1686,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,10,300791,182,Paul,Coronaviney,5,0,5,15839,16167,20,4
0,10,8258,9856,Simon,It's always Sonny,4,0,6,16259,15947,16,5
0,10,10857,189,Anthony,The Full Bonty,4,0,6,16282,16115,16,6
0,10,21562,186,Mark,GetRichOrDieTryin,4,0,6,16257,16397,16,7


In [11]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [12]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476
0,1,James vs Jordan,9867,Bad Boys For Fyfe,James,1623,33301,"What's Up, Doch?",Jordan,1721
0,2,Paul vs Anthony,182,Coronaviney,Paul,1676,189,The Full Bonty,Anthony,1653
0,2,Lester vs Mark,9806,HoweIMetYourMother,Lester,1658,186,GetRichOrDieTryin,Mark,1642
0,2,James vs Luke,9867,Bad Boys For Fyfe,James,1538,5524,The Dragon's Laird,Luke,1906
0,2,Jordan vs Simon,33301,"What's Up, Doch?",Jordan,1700,9856,It's always Sonny,Simon,1659
0,3,Paul vs Lester,182,Coronaviney,Paul,1590,9806,HoweIMetYourMother,Lester,1679
0,3,James vs Anthony,9867,Bad Boys For Fyfe,James,1263,189,The Full Bonty,Anthony,1411


In [13]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [14]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476,1.0,8.0
0,1,James vs Jordan,33301,"What's Up, Doch?",Jordan,1721,9867,Bad Boys For Fyfe,James,1623,2.0,7.0
0,1,Anthony vs Luke,5524,The Dragon's Laird,Luke,1720,189,The Full Bonty,Anthony,1662,3.0,5.0
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657,4.0,6.0
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,10,Paul vs Lester,182,Coronaviney,Paul,1583,9806,HoweIMetYourMother,Lester,1548,4.0,5.0
0,10,Paul vs Lester,9806,HoweIMetYourMother,Lester,1548,182,Coronaviney,Paul,1583,5.0,4.0
0,10,James vs Anthony,9867,Bad Boys For Fyfe,James,1516,189,The Full Bonty,Anthony,1792,6.0,1.0
0,10,Jordan vs Mark,186,GetRichOrDieTryin,Mark,1437,33301,"What's Up, Doch?",Jordan,1598,7.0,3.0


## CURRENTLY LISTED TEAMS

In [15]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,151,owner,186.0,NaN,293957,Brodie,Grundy,COL,RUC,21562,186,Mark,GetRichOrDieTryin
1,469,owner,33301.0,NaN,290528,Max,Gawn,MEL,RUC,180650,33301,Jordan,"What's Up, Doch?"
2,755,owner,33301.0,NaN,295467,Jack,Macrae,WBD,MID,180650,33301,Jordan,"What's Up, Doch?"
3,76,owner,5524.0,NaN,293535,Lachie,Neale,BRL,MID,45168,5524,Luke,The Dragon's Laird
4,244,owner,9867.0,NaN,291570,Nat,Fyfe,FRE,MID,71052,9867,James,Bad Boys For Fyfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,598,owner,189.0,NaN,1002245,Noah,Balta,RIC,RUC FWD,10857,189,Anthony,The Full Bonty
172,174,owner,182.0,NaN,1003029,Brayden,Sier,COL,MID,300791,182,Paul,Coronaviney
173,645,owner,33301.0,NaN,992374,Dan,Butler,STK,FWD,180650,33301,Jordan,"What's Up, Doch?"
174,263,owner,9806.0,NaN,1009420,Caleb,Serong,FRE,MID,86078,9806,Lester,HoweIMetYourMother


## CURRENT FREE AGENTS + WAIVER

In [16]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position
32,513,free_agent,NaN,NaN,280921,Ben,Cunnington,NTH,MID
43,5,free_agent,NaN,NaN,294307,Brad,Crouch,ADE,MID
45,676,free_agent,NaN,NaN,293846,Sebastian,Ross,STK,MID
47,108,free_agent,NaN,NaN,261299,Matthew,Kreuzer,CAR,RUC
53,116,free_agent,NaN,NaN,297907,Nic,Newman,CAR,DEF
...,...,...,...,...,...,...,...,...,...
821,818,free_agent,NaN,NaN,1016214,Ayce,Taylor,ADE,DEF
822,821,free_agent,NaN,NaN,1016097,Henry,Crauford,ESS,RUC
823,822,free_agent,NaN,NaN,1020224,Tom,Hird,ESS,MID
824,825,free_agent,NaN,NaN,297668,Nicholas,Reid,WCE,FWD


In [17]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [65]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486,291533,Tom,McDonald,MEL
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577,293987,Jarrod,Lienert,PTA
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802,281373,Jack,Redden,WCE
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356,298419,Brandan,Parfitt,GEE
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278,998129,Will,Brodie,GCS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11,186,2020-08-08 04:00:01,454,280737,Liam,Shiels,HAW,549,998484,Cameron,Zurhaar,NTH
96,11,182,2020-08-08 04:00:01,795,240406,Josh J.,Kennedy,WCE,203,298630,Kyle,Langford,ESS
97,11,182,2020-08-08 04:00:01,235,1002239,Adam,Cerra,FRE,689,291978,Harry,Cunningham,SYD
98,11,9856,2020-08-10 04:00:01,34,294733,Paul,Seedsman,ADE,403,998205,Harry,Perryman,GWS


In [67]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [64]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev
0,2,182,2020-06-11 18:47:54,606,270896,Trent,Cotchin,RIC,675.0,280933,Dylan,Roberton,STK
1,3,182,2020-06-21 19:41:30,564,1006096,Xavier,Duursma,PTA,579.0,1004998,Todd,Marshall,PTA
2,3,182,2020-06-21 19:39:37,550,297354,Karl,Amon,PTA,214.0,993817,Darcy,Parish,ESS
3,3,182,2020-06-20 23:20:43,579,1004998,Todd,Marshall,PTA,227.0,296294,Blake,Acres,FRE
4,3,182,2020-06-20 15:04:59,228,295067,James,Aish,FRE,199.0,270588,Cale,Hooker,ESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,10,33301,2020-08-03 05:50:12,379,294508,Aidan,Corr,GWS,167.0,1008089,Isaac,Quaynor,COL
143,10,33301,2020-08-04 09:13:35,597,1000223,Liam,Baker,RIC,609.0,260930,Shane,Edwards,RIC
144,11,33301,2020-08-08 04:36:33,575,292145,Tom,Jonas,PTA,693.0,291848,Robbie,Fox,SYD
145,11,33301,2020-08-08 12:43:59,507,290787,Shaun,Atley,NTH,379.0,294508,Aidan,Corr,GWS


In [66]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Combined Free Agents and Waiver transactions list

In [75]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486.0,291533,Tom,McDonald,MEL
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577.0,293987,Jarrod,Lienert,PTA
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802.0,281373,Jack,Redden,WCE
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356.0,298419,Brandan,Parfitt,GEE
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278.0,998129,Will,Brodie,GCS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,11,9806,2020-08-09 15:30:30,514,1002267,Luke,Davies-Uniacke,NTH,413.0,280109,Callan,Ward,GWS
99,11,182,2020-08-10 04:00:01,203,298630,Kyle,Langford,ESS,274.0,998130,Ben,Ainsworth,GCS
98,11,9856,2020-08-10 04:00:01,34,294733,Paul,Seedsman,ADE,403.0,998205,Harry,Perryman,GWS
113,11,9856,2020-08-10 17:40:38,416,280744,Luke,Breust,HAW,277.0,998260,Jack,Bowes,GCS


In [76]:
df_transactions.to_csv('outputs/transactions.csv')

## Team Trades

In [69]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)
df_team_trade_transactions = pd.json_normalize(team_trades_json, max_level=0)
df_team_trade_transactions
# wanted_columns = [
#     'round',
#     'user_id',
#     'action_time',
#     'buy_player.id',
#     'buy_player.feed_id',
#     'buy_player.first_name',
#     'buy_player.last_name',
#     'buy_player.team.abbrev',
#     'sell_player.id',
#     'sell_player.feed_id',
#     'sell_player.first_name',
#     'sell_player.last_name',
#     'sell_player.team.abbrev',
# ]

# df_team_trade_transactions = df_team_trade_transactions.loc[:, wanted_columns]
# df_team_trade_transactions

,id,round,status_id,league_id,user_team_id,trade_user_team_id,votes,action_time,team_trade_players,team_trade_drop_players,team_trade_votes
0,16727,2,2,96,182,189,0,2020-06-12 10:01:48,"[{'id': 26827, 'trade_id': 16727, 'player_id':...",[],[]
1,19019,3,7,96,186,9856,0,2020-06-15 20:17:51,"[{'id': 30776, 'trade_id': 19019, 'player_id':...",[],[]
2,24785,4,3,96,182,9867,0,2020-07-07 00:31:22,"[{'id': 40668, 'trade_id': 24785, 'player_id':...",[],[]
3,30082,5,3,96,33301,9856,0,2020-06-30 21:43:08,"[{'id': 49794, 'trade_id': 30082, 'player_id':...",[],[]
4,30167,5,2,96,5524,33301,0,2020-07-01 19:11:42,"[{'id': 49956, 'trade_id': 30167, 'player_id':...",[],[]
5,30177,5,2,96,5524,33301,0,2020-06-30 22:33:56,"[{'id': 49974, 'trade_id': 30177, 'player_id':...",[],[]
6,31052,5,7,96,9856,33301,0,2020-07-01 21:22:52,"[{'id': 51546, 'trade_id': 31052, 'player_id':...","[{'id': 680, 'trade_id': 31052, 'user_team_id'...",[]
7,34214,6,3,96,182,9806,0,2020-07-08 19:53:04,"[{'id': 57022, 'trade_id': 34214, 'player_id':...",[],[]
8,35338,6,3,96,189,182,0,2020-07-13 14:40:19,"[{'id': 59004, 'trade_id': 35338, 'player_id':...",[],[]
9,35641,6,2,96,182,9806,0,2020-07-09 01:18:32,"[{'id': 59547, 'trade_id': 35641, 'player_id':...",[],[]
